# Z3 SAT Constraints Encodings

In this notebook we are going to show several implementation in z3 of the cardinality constraints and show how they affect the efficiency of the solver. 

In [6]:
from z3 import *
import math

## At most one encodings

### Naive pairwise

In [2]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars):
    return at_most_one_np(bool_vars) + [at_least_one_np(bool_vars)]

### Sequential 

In [3]:
def at_least_one_seq(bool_vars):
    return at_most_one_seq([Not(var) for var in bool_vars])

def at_most_one_seq(bool_vars):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{i}") for i in range(n)]
    for i in range(n):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), s[i-1]))
        if i > 0 and i < n - 1:
            constraints.append(Or(Not(s[i-1]), s[i]))
    return constraints

def exactly_one_seq(bool_vars):
    return at_most_one_seq(bool_vars) + at_least_one_seq(bool_vars)

### Bitwise

In [11]:
def at_least_one_bw(bool_vars):
    return at_most_one_b([Not(var) for var in bool_vars])

def at_most_one_bw(bool_vars):
    constraints = []
    n = len(bool_vars)
    m = int(math.log2(n - 1))
    r = [Bool(f"r_{i}") for i in range(m)]
    for i in range(n):
        for j in range(m):
            i_binary = bin(i)[2:]
            for k in i_binary:
                if k == "1": 
                    constraints.append(Or(Not(bool_vars[i]), r[j]))
                else:
                    constraints.append(Or(Not(bool_vars[i]), Not(r[j])))
    return constraints

def exactly_one_bw(bool_vars):
    return at_most_one_bw(bool_vars) + at_least_one_bw(bool_vars)

### Heule 

In [12]:
def at_least_one_he(bool_vars):
    return at_most_one_he([Not(var) for var in bool_vars])

def at_most_one_he(bool_vars):
    y = Bool("y")
    return at_most_one_np(bool_vars[:3] + [y]) + at_most_one_np(bool_vars[3:] + [Not(y)])

def exactly_one_he(bool_vars):
    return at_most_one_he(bool_vars) + at_least_one_he(bool_vars)

## At most k encodings

### Naive pairwise

In [13]:
def at_least_k_np(bool_vars, k):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_np(bool_vars, k):
    constraints = []
    for i in range(k):
        constraints+=[And([Not(var) if var not in pair else var for var in bool_vars]) for pair in combinations(bool_vars, i + 1)]  
    return Or(constraints)

def exactly_k_np(bool_vars, k):
    return [at_most_k_np(bool_vars, k)] + [at_least_k_np(bool_vars, k)]

### Sequential

In [ ]:
def at_least_k_seq(bool_vars, k):
    return at_most_k_seq([Not(var) for var in bool_vars], k)

def at_most_k_seq(bool_vars, k):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{i}_{j}") for i in range(n) for j in range(k)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][-1])))
        for j in range(1, k):
            constraints.append(Or([Not(bool_vars[i])] + [(Not(s[i-1][j-1]))] + [s[i][j]]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[-1]), Not(s[-1][-1])))   
    return constraints

def exactly_k_seq(bool_vars, k):
    return at_most_k_seq(bool_vars, k) + at_least_k_seq(bool_vars, k)